In [1]:
import time
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import pyupbit

# --- 전처리 함수 ---
def rolling_minmax_scale(series, window=24):
    roll_min = series.rolling(window=window, min_periods=window).min()
    roll_max = series.rolling(window=window, min_periods=window).max()
    scaled = (series - roll_min) / ((roll_max - roll_min) + 1e-8)
    scaled = scaled.replace([np.inf, -np.inf], np.nan)
    scaled = scaled.fillna(1.0)
    return scaled.clip(upper=1.0)

def bin_and_encode(data, features, bins=100, drop_original=True):
    for feature in features:
        data[f'{feature}_Bin'] = pd.cut(data[feature], bins=bins, labels=False)
        one_hot = pd.get_dummies(data[f'{feature}_Bin'], prefix=f'{feature}_Bin').astype(np.int32)
        expected_columns = [f'{feature}_Bin_{i}' for i in range(bins)]
        one_hot = one_hot.reindex(columns=expected_columns, fill_value=0)
        data = pd.concat([data, one_hot], axis=1)
        if drop_original:
            data.drop(columns=[f'{feature}_Bin'], inplace=True)
    # 모든 숫자형 열을 float32로 변환
    numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        data[col] = data[col].astype(np.float32)
    return data

# --- 모델 정의 ---
class EncoderOnlyTransformerCustom(nn.Module):
    def __init__(self, input_dim, embedding_dim=512, num_layers=6, nhead=8, ffn_dim=2048, num_classes=2, max_seq_len=24):
        super(EncoderOnlyTransformerCustom, self).__init__()
        self.token_embedding = nn.Linear(input_dim, embedding_dim)
        self.position_embedding = nn.Embedding(max_seq_len, embedding_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=nhead, dim_feedforward=ffn_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embedding_dim, num_classes)
        
    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        x = self.token_embedding(x)
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0).expand(batch_size, seq_len)
        pos_emb = self.position_embedding(positions)
        x = x + pos_emb
        x = x.transpose(0, 1)  # Transformer의 입력형태: [seq_len, batch, features]
        x = self.transformer_encoder(x)
        return self.fc(x[-1, :, :])  # 마지막 타임스탭의 출력을 사용

# --- 모델 로드 ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# OHLC 4개 feature를 100구간으로 binning했으므로, 입력 차원은 4*100 = 400
input_dim = 400
model = EncoderOnlyTransformerCustom(input_dim=input_dim).to(device)
model_path = "model_experiment_15.pth"  # 모델 파일 경로 (실제 파일명에 맞게 수정)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# --- pyupbit API 설정 ---
access = "x03uKM3uyZ9RNDBHLxauEaeI6X8pdKQq8rHG2wvx"
secret = "e9UyPqJBjCvNDxCmtx7bdPaHtAEgw2Ftxu3UpuJc"
upbit = pyupbit.Upbit(access, secret)
ticker = "KRW-ETH"  # 거래할 코인 티커

# --- 예측 함수 ---
def get_model_prediction():
    """
    pyupbit를 이용해 최근 50개의 5분봉 데이터를 가져와 전처리한 후,
    마지막 24개 봉을 모델 입력으로 하여 다음 5분 상승/하락 예측(1: 상승, 0: 하락)을 반환.
    """
    df = pyupbit.get_ohlcv(ticker, interval="minute5", count=50)
    if df is None or len(df) < 24:
        print("예측에 충분한 데이터가 없습니다.")
        return None
    ohlc_features = ['open', 'high', 'low', 'close']
    for feature in ohlc_features:
        df[feature] = rolling_minmax_scale(df[feature], window=24)
    df_processed = bin_and_encode(df.copy(), ohlc_features, bins=100, drop_original=True)
    # _Bin_ 문자열이 포함된 열만 선택
    final_input_columns = [col for col in df_processed.columns if '_Bin_' in col]
    if len(df_processed) < 24:
        print("lookback 데이터가 부족합니다.")
        return None
    input_seq = df_processed[final_input_columns].iloc[-24:]
    # 모델 입력 형태: [batch, seq_len, features]
    x = torch.tensor(input_seq.values, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(x)
        prediction = torch.argmax(outputs, dim=1).item()  # 1이면 상승, 0이면 하락
    return prediction

# --- 거래 함수 ---
def trade_decision():
    prediction = get_model_prediction()
    if prediction is None:
        return
    print(f"모델 예측 결과 (1: 상승, 0: 하락): {prediction}")
    if prediction == 1:
        # 상승 예측 시: 보유 KRW의 1%로 매수
        krw_balance = upbit.get_balance("KRW")
        if krw_balance is None or krw_balance < 5000:
            print("매수할 충분한 KRW 잔고가 없습니다.")
            return
        trade_amount = krw_balance * 0.2
        print(f"{ticker}을(를) {trade_amount:.0f} KRW로 매수합니다.")
        upbit.buy_market_order(ticker, trade_amount)
    else:
        # 하락 예측 시: 보유 코인의 1% 매도
        coin = ticker.split("-")[1]
        coin_balance = upbit.get_balance(coin)
        if coin_balance is None or coin_balance <= 0:
            print("매도할 보유 코인이 없습니다.")
            return
        trade_amount = coin_balance * 0.2
        print(f"{coin}의 {trade_amount:.4f}개를 매도합니다.")
        upbit.sell_market_order(ticker, trade_amount)

# --- 메인 루프: 새로운 5분봉이 생성될 때마다 업데이트 ---
last_candle_time = None
while True:
    try:
        # 최신 5분봉 데이터 1개 (마지막 봉)
        df = pyupbit.get_ohlcv(ticker, interval="minute5", count=1)
        if df is not None and not df.empty:
            current_candle_time = df.index[-1]
            # 새로운 봉이 생성되었으면 trade_decision() 실행
            if last_candle_time is None or current_candle_time > last_candle_time:
                last_candle_time = current_candle_time
                print(f"새로운 5분봉 생성: {last_candle_time}")
                trade_decision()
        else:
            print("최신 5분봉 데이터를 불러오지 못했습니다.")
    except Exception as e:
        print(f"에러 발생: {e}")
    # 10초마다 확인 (필요에 따라 sleep 시간을 조정)
    time.sleep(10)


c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


새로운 5분봉 생성: 2025-03-10 01:40:00


c:\Users\User\anaconda3\envs\py39coin\lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


모델 예측 결과 (1: 상승, 0: 하락): 0
TypeError
매도할 보유 코인이 없습니다.
새로운 5분봉 생성: 2025-03-10 01:45:00
모델 예측 결과 (1: 상승, 0: 하락): 0
TypeError
매도할 보유 코인이 없습니다.
새로운 5분봉 생성: 2025-03-10 01:50:00
모델 예측 결과 (1: 상승, 0: 하락): 0
TypeError
매도할 보유 코인이 없습니다.
새로운 5분봉 생성: 2025-03-10 01:55:00
모델 예측 결과 (1: 상승, 0: 하락): 0
TypeError
매도할 보유 코인이 없습니다.
새로운 5분봉 생성: 2025-03-10 02:00:00
모델 예측 결과 (1: 상승, 0: 하락): 0
TypeError
매도할 보유 코인이 없습니다.
새로운 5분봉 생성: 2025-03-10 02:05:00
모델 예측 결과 (1: 상승, 0: 하락): 0
TypeError
매도할 보유 코인이 없습니다.
새로운 5분봉 생성: 2025-03-10 02:10:00
모델 예측 결과 (1: 상승, 0: 하락): 0
TypeError
매도할 보유 코인이 없습니다.
새로운 5분봉 생성: 2025-03-10 02:15:00
모델 예측 결과 (1: 상승, 0: 하락): 0
TypeError
매도할 보유 코인이 없습니다.
새로운 5분봉 생성: 2025-03-10 02:20:00
모델 예측 결과 (1: 상승, 0: 하락): 0
TypeError
매도할 보유 코인이 없습니다.
새로운 5분봉 생성: 2025-03-10 02:25:00
모델 예측 결과 (1: 상승, 0: 하락): 0
TypeError
매도할 보유 코인이 없습니다.
새로운 5분봉 생성: 2025-03-10 02:30:00
모델 예측 결과 (1: 상승, 0: 하락): 0
TypeError
매도할 보유 코인이 없습니다.
새로운 5분봉 생성: 2025-03-10 02:35:00
모델 예측 결과 (1: 상승, 0: 하락): 0
TypeError
매도할 보유 코인이 없습니다.


KeyboardInterrupt: 

In [1]:
import time
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import pyupbit

# --- 전처리 함수 ---
def rolling_minmax_scale(series, window=6):
    roll_min = series.rolling(window=window, min_periods=window).min()
    roll_max = series.rolling(window=window, min_periods=window).max()
    scaled = (series - roll_min) / ((roll_max - roll_min) + 1e-8)
    scaled = scaled.replace([np.inf, -np.inf], np.nan)
    scaled = scaled.fillna(1.0)
    return scaled.clip(upper=1.0)

def bin_and_encode(data, features, bins=100, drop_original=True):
    for feature in features:
        data[f'{feature}_Bin'] = pd.cut(data[feature], bins=bins, labels=False)
        one_hot = pd.get_dummies(data[f'{feature}_Bin'], prefix=f'{feature}_Bin').astype(np.int32)
        expected_columns = [f'{feature}_Bin_{i}' for i in range(bins)]
        one_hot = one_hot.reindex(columns=expected_columns, fill_value=0)
        data = pd.concat([data, one_hot], axis=1)
        if drop_original:
            data.drop(columns=[f'{feature}_Bin'], inplace=True)
    numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        data[col] = data[col].astype(np.float32)
    return data

# --- Diffusion Model 구성 ---
class ConditionEncoder(nn.Module):
    def __init__(self, input_dim, lookback, condition_dim):
        super(ConditionEncoder, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim * lookback, condition_dim),
            nn.ReLU(),
            nn.Linear(condition_dim, condition_dim)
        )
        
    def forward(self, x):
        # x: [batch, lookback, input_dim]
        batch_size = x.size(0)
        x = x.contiguous().view(batch_size, -1)
        return self.fc(x)

class DiffusionClassifier(nn.Module):
    def __init__(self, input_dim, lookback, condition_dim=128, num_timesteps=100, hidden_dim=128):
        super(DiffusionClassifier, self).__init__()
        self.num_timesteps = num_timesteps
        # diffusion 스케줄 (선형 beta schedule)
        betas = torch.linspace(1e-4, 0.02, num_timesteps)
        alphas = 1 - betas
        alphas_cumprod = torch.cumprod(alphas, dim=0)
        self.register_buffer('betas', betas)
        self.register_buffer('alphas', alphas)
        self.register_buffer('alphas_cumprod', alphas_cumprod)
        
        # 조건 인코더: 시계열 window를 임베딩
        self.condition_encoder = ConditionEncoder(input_dim, lookback, condition_dim)
        # timestep 임베딩
        self.time_embedding = nn.Embedding(num_timesteps, hidden_dim)
        # 노이즈 예측 네트워크
        self.model = nn.Sequential(
            nn.Linear(1 + condition_dim + hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, x_condition, y_noisy, t):
        # x_condition: [batch, lookback, input_dim]
        # y_noisy: [batch, 1]
        # t: [batch]
        cond = self.condition_encoder(x_condition)           # [batch, condition_dim]
        t_emb = self.time_embedding(t)                         # [batch, hidden_dim]
        inp = torch.cat([y_noisy, cond, t_emb], dim=1)          # [batch, 1 + condition_dim + hidden_dim]
        predicted_noise = self.model(inp)                      # [batch, 1]
        return predicted_noise
    
    def sample(self, x_condition, device):
        """
        reverse diffusion 과정을 통해 조건 x_condition에 대해 예측값을 샘플링.
        최종 출력은 continuous 값으로, 임계값 0.5를 기준으로 상승/하락을 결정.
        """
        batch_size = x_condition.size(0)
        # 초기 y: 정규분포 노이즈
        y = torch.randn(batch_size, 1, device=device)
        for t in reversed(range(self.num_timesteps)):
            t_tensor = torch.full((batch_size,), t, device=device, dtype=torch.long)
            predicted_noise = self.forward(x_condition, y, t_tensor)
            alpha = self.alphas[t]
            alpha_cumprod = self.alphas_cumprod[t]
            beta = self.betas[t]
            y = (1 / torch.sqrt(alpha)) * (y - (beta / torch.sqrt(1 - alpha_cumprod)) * predicted_noise)
            if t > 0:
                noise = torch.randn_like(y)
                y = y + torch.sqrt(beta) * noise
        return y

# --- 모델 로드 ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# OHLC 4개 feature를 100구간으로 binning했으므로 입력 차원은 4*100 = 400
input_dim = 400
lookback = 6
model = DiffusionClassifier(input_dim=input_dim, lookback=lookback, condition_dim=128, num_timesteps=100, hidden_dim=128).to(device)
model_path = "diffusion_model_experiment_14_6.pth"  # 실제 학습된 모델 파일 경로로 수정
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# --- pyupbit API 설정 ---
access = "x03uKM3uyZ9RNDBHLxauEaeI6X8pdKQq8rHG2wvx"
secret = "e9UyPqJBjCvNDxCmtx7bdPaHtAEgw2Ftxu3UpuJc"
upbit = pyupbit.Upbit(access, secret)
ticker = "KRW-ETH"  # 거래할 코인 티커

# --- 예측 함수 ---
def get_model_prediction():
    """
    pyupbit로 최근 50개의 5분봉 데이터를 가져와 전처리한 후,
    마지막 24개 봉을 diffusion model의 조건으로 사용하여
    reverse diffusion sampling을 통해 다음 5분봉 상승/하락 예측(1: 상승, 0: 하락)을 반환.
    """
    df = pyupbit.get_ohlcv(ticker, interval="minute5", count=15)
    if df is None or len(df) < lookback:
        print("예측에 충분한 데이터가 없습니다.")
        return None
    ohlc_features = ['open', 'high', 'low', 'close']
    for feature in ohlc_features:
        df[feature] = rolling_minmax_scale(df[feature], window=6)
    df_processed = bin_and_encode(df.copy(), ohlc_features, bins=100, drop_original=True)
    final_input_columns = [col for col in df_processed.columns if '_Bin_' in col]
    if len(df_processed) < lookback:
        print("lookback 데이터가 부족합니다.")
        return None
    input_seq = df_processed[final_input_columns].iloc[-lookback:]
    # 모델 입력 형태: [batch, seq_len, features]
    x = torch.tensor(input_seq.values, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        y_cont = model.sample(x, device)  # continuous output
        prediction = 1 if y_cont.item() >= 0.5 else 0
    return prediction

# --- 거래 함수 ---
def trade_decision():
    prediction = get_model_prediction()
    if prediction is None:
        return
    print(f"모델 예측 결과 (1: 상승, 0: 하락): {prediction}")
    if prediction == 1:
        # 상승 예측 시: 보유 KRW의 20%로 매수
        krw_balance = upbit.get_balance("KRW")
        if krw_balance is None or krw_balance < 5000:
            print("매수할 충분한 KRW 잔고가 없습니다.")
            return
        trade_amount = krw_balance * 0.5
        print(f"{ticker}을(를) {trade_amount:.0f} KRW로 매수합니다.")
        upbit.buy_market_order(ticker, trade_amount)
    else:
        # 하락 예측 시: 보유 코인의 20% 매도
        coin = ticker.split("-")[1]
        coin_balance = upbit.get_balance(coin)
        if coin_balance is None or coin_balance <= 0:
            print("매도할 보유 코인이 없습니다.")
            return
        trade_amount = coin_balance * 0.5
        print(f"{coin}의 {trade_amount:.4f}개를 매도합니다.")
        upbit.sell_market_order(ticker, trade_amount)

# --- 메인 루프: 새로운 5분봉이 생성될 때마다 업데이트 ---
last_candle_time = None
while True:
    try:
        # 최신 5분봉 데이터 1개 (마지막 봉)
        df = pyupbit.get_ohlcv(ticker, interval="minute5", count=1)
        if df is not None and not df.empty:
            current_candle_time = df.index[-1]
            # 새로운 봉이 생성되었으면 trade_decision() 실행
            if last_candle_time is None or current_candle_time > last_candle_time:
                last_candle_time = current_candle_time
                print(f"새로운 5분봉 생성: {last_candle_time}")
                trade_decision()
        else:
            print("최신 5분봉 데이터를 불러오지 못했습니다.")
    except Exception as e:
        print(f"에러 발생: {e}")
    # 10초마다 확인 (필요에 따라 sleep 시간을 조정)
    time.sleep(10)


새로운 5분봉 생성: 2025-03-16 11:05:00
모델 예측 결과 (1: 상승, 0: 하락): 1
KRW-ETH을(를) 36162 KRW로 매수합니다.


KeyboardInterrupt: 